## 1. Install and Import Dependencies

In [1]:
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
import re
from bs4 import BeautifulSoup

## 2. Instantiate Model

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

## 3. Encode and Calculate Sentiment

In [5]:
tokens = tokenizer.encode('It was awesome, I loved it', return_tensors='pt')

In [6]:
tokens[0]

tensor([  101, 10197, 10140, 37079, 42279, 10688,   117,   151, 46747, 10197,
          102])

In [7]:
tokenizer.decode(tokens[0])

'[CLS] it was awesome, i loved it [SEP]'

In [8]:
result = model(tokens)

In [9]:
result.logits

tensor([[-2.3924, -2.3591, -0.3630,  1.5136,  2.8604]],
       grad_fn=<AddmmBackward0>)

In [10]:
torch.argmax(result.logits)

tensor(4)

In [11]:
int(torch.argmax(result.logits))+1

5

## 4. Collect Reviews

In [12]:
r = requests.get('https://www.yelp.com/biz/mcdonalds-new-york-386?osq=McDonald%27s')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})

In [13]:
results

[<p class="comment__09f24__D0cxf y-css-h9c2fl"><span class="raw__09f24__T4Ezm" lang="en">Been a regular at this McDonald's ever since 2003 since it was a couple of blocks away from my high school. It is considered the McDonald's of the Financial District or Wall Street McDonald's for me. They remodeled and renovated this McDonald's so now they lost the old touch which I missed. This McDonald's used to have a piano, a fancy staircase to go up to the 2nd floor for dine in. There was also a showcase of McDonald's memorabilia but I don't see that anymore. Now it is more modern and have a pick up section along with many kiosks to do self ordering which is the norm these days for anyone who don't want to talk to people.<br/><br/>This time when I got to McDonald's I ordered a double Big Mac which consists of 4 patties! Like always, the Big Mac wins my heart and never fail to satisfy me. I would forever love Big Mac and I don't think anyone or anything can stop me from loving it! It's my favor

In [14]:
reviews = [result.text for result in results]

In [15]:
reviews[0]

"Been a regular at this McDonald's ever since 2003 since it was a couple of blocks away from my high school. It is considered the McDonald's of the Financial District or Wall Street McDonald's for me. They remodeled and renovated this McDonald's so now they lost the old touch which I missed. This McDonald's used to have a piano, a fancy staircase to go up to the 2nd floor for dine in. There was also a showcase of McDonald's memorabilia but I don't see that anymore. Now it is more modern and have a pick up section along with many kiosks to do self ordering which is the norm these days for anyone who don't want to talk to people.This time when I got to McDonald's I ordered a double Big Mac which consists of 4 patties! Like always, the Big Mac wins my heart and never fail to satisfy me. I would forever love Big Mac and I don't think anyone or anything can stop me from loving it! It's my favorite item on the menu. The sauce that is in the sandwich is what makes it so delicious!  French fri

## 5. Load Reviews into DataFrame and Score

In [16]:
import pandas as pd
import numpy as np

In [17]:
df = pd.DataFrame(np.array(reviews), columns = ['review'])

In [18]:
df

,review
0,Been a regular at this McDonald's ever since 2...
1,Quick pick up and delivery location.. kind of ...
2,I hate this pLACE THERE was a rat in my carame...
3,I loved this place! I got a coke slushy and th...
4,"The food comes out in a very timely manner, wi..."
5,This McDonald is good lactation. It is near T...
6,Worst place I've ever been to. I've been here...
7,Nobody speaks proper English they all speak Sp...
8,"Perfect meal, perfect food for my family.This ..."
9,found a roach in my big mac the chicken nugget...


In [19]:
df['review'].iloc[0]

"Been a regular at this McDonald's ever since 2003 since it was a couple of blocks away from my high school. It is considered the McDonald's of the Financial District or Wall Street McDonald's for me. They remodeled and renovated this McDonald's so now they lost the old touch which I missed. This McDonald's used to have a piano, a fancy staircase to go up to the 2nd floor for dine in. There was also a showcase of McDonald's memorabilia but I don't see that anymore. Now it is more modern and have a pick up section along with many kiosks to do self ordering which is the norm these days for anyone who don't want to talk to people.This time when I got to McDonald's I ordered a double Big Mac which consists of 4 patties! Like always, the Big Mac wins my heart and never fail to satisfy me. I would forever love Big Mac and I don't think anyone or anything can stop me from loving it! It's my favorite item on the menu. The sauce that is in the sandwich is what makes it so delicious!  French fri

In [20]:
df.iloc[0]

review    Been a regular at this McDonald's ever since 2...
Name: 0, dtype: object

In [21]:
def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits))+1

In [22]:
sentiment_score(df['review'].iloc[0])

5

In [23]:
df['sentiment_score'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [24]:
df

,review,sentiment_score
0,Been a regular at this McDonald's ever since 2...,5
1,Quick pick up and delivery location.. kind of ...,4
2,I hate this pLACE THERE was a rat in my carame...,1
3,I loved this place! I got a coke slushy and th...,5
4,"The food comes out in a very timely manner, wi...",4
5,This McDonald is good lactation. It is near T...,4
6,Worst place I've ever been to. I've been here...,1
7,Nobody speaks proper English they all speak Sp...,1
8,"Perfect meal, perfect food for my family.This ...",5
9,found a roach in my big mac the chicken nugget...,1


## Save the DataFrame

In [27]:
df.to_csv('Reviews.csv', index=False)